# I/O and File System

```c
#include <stdio.h>
int main(void){
    FILE *fp = fopen("output.txt","w");
    if (!fp){
        perror("output.txt");
        return 1;
    }
    fputs("baby shark (do doo dooo)\n", fp);
    if(fclose(fp)){
        perror("output.txt");
        return 1;
    }
    return 0;
}

FILE *fopen(const char *fname, const char *mode){
    int fd = open(fnam, __mode2flags(mode), DEFFILEPERMS);
    if (fd == -1){
        return NULL;
    }
    return fdopen(fd, mode);
}

int fclose (FILE *fp){
    int rv = close(fp->fd);
    __ffree(fp);
    return rv;
}

```

```asm
.globl close
close:
    mov $3, %eax
    syscall
    cmp $-4096, %rax
    jae __syscall_error
    ret
```

## Unix I/O

A Linux *file* is a sequence of `m` bytes:
* $B_0, B_1, ... , B_k, ... , B_{m-1}$
* Each *file* has some kind of low-level name: inode (index node) number
  * inode points to where the data of the file is stored

All I/O devices are represented as file:
* `/dev/tty2` (Terminal)
* `/dev/sda2` (Disk Partition)

Kernel data structures are exposed as files  
* `cat /proc/$$/status` (How long it has been running)
* `ls -l /proc/$$/fd/`

### Types
Two Abstractions: File and Directory
* File:
  * filename: human-readable
  * low-level name: inode number
* Directory
  * low-level name: inode number
  * contents are quite specific: pairs of (filename, inode number) or (subdirectory, inode number)

Elegant mapping of files to devices allows kernel to export simple interface called Unix 1/0:
* Opening and closing files
  * `open()` and `close()`
* Reading and writing a file
  * `read()` and `write()`
* Changing the *current file position* (seek)
  * indicated next offset into file to read or write
  * `lseek()`

### Directories
Directory consists of array of links
* Each link maps a filename to a file
* Pairs of (filename, inode number)

Each directory contains at least two entries
* `.` (dot) is a link to itself
* `..` (dot dot) is a link to the parent directory in the directory hierarchy

Commands for manipulating directories
* `mkdir`: create empty
* `ls`: list contents of directory
* `rmdir`: remove directory

All files are organized as a hierarchy anchored by root directory named `/`

Kernel maintains *current working directory* `cwd`

Locations of files in the hierarchy denoted by *pathnames*
* Aboslute pathname starts with `/` and denotes path from root
* Relative pathname denotes path from current working directory

### Opening Files
Opening a file informs the kernel that uou are getting ready to access that file

```c
int fd; // file descriptor

if ((fd = open("/etc/hosts", O_RDONLY)) < 0){
    perror("open");
...
```

file descriptor:
* an integer
* like a pointer to an object of type file; you can call methods to this
* like a “pointer” to an object of type file; you can call “methods” to access the file: read(), write() 

When you first open a file, the file descriptor is 3

Each running process already has three files open:
* standard input (which the process can read to receive input)
* standard output (which the process can write to in order to dump information to the screen)
* standard error (which the process can write error messages to)
* These are represented by file descriptors 0, 1, and 2, respectively

### Closing files
Closing a file informs the kernel that you are finished accessing that file
```c
if (close(fd) < 0) {
   fprintf(stderr, "%s: write error: %s",
exit(1);filename, strerror(errno));
}
```
Moral: Always check return codes, even for seemingly benign functions such as close()

### Reading Files
Reading a file copies bytes from the current file position, and then updates file position
```c
char buf[512];
int fd;       /* file descriptor */
int nbytes;   /* number of bytes read */
/* Open file fd ...  */
/* Then read up to 512 bytes from file fd */
if ((nbytes = read(fd, buf, sizeof(buf))) < 0) {
}
perror("read");
exit(1);
```
Returns the number of bytes read from file `fd` into `buf`
* Return types size_t is signed integer
* nbytes < 0 indicates that an error occurred
* Shortcounts`(nbytes<sizeof(buf))` are possible and are not errors!

### Writing Files
Writing a file copies bytes to the current file position, and then updates current file position
```c
char buf[512];
int fd;       /* file descriptor */
int nbytes;   /* number of bytes read */
/* Open the file fd ... */
/* Then write up to 512 bytes from buf to file fd */ if ((nbytes = write(fd, buf, sizeof(buf)) < 0) {
}
perror("write");
exit(1);
```
Returns number of bytes written from `buf` to file `fd`
* nbytes < 0 indicates that an error occurred  
* As with reads, short counts are possible and are not errors!

### File Metadata
Each file has an inode that stores *Metadata* (data about file data). This is maintained by kernel

### Practice
1.	What is the output of the following program?
```c
int main(){
    int fd1, fd2;
    fd1 = open(“foo.txt”, O_RDONLY, 0);
    close(fd1);
    fd2 = open(‘baz.txt’, O_RDONLY, 0);
    printf(“fd2 = %d\n”, fd2);
    exit(0);
} 
```
Output: `3`

2.	Suppose the disk file foobar.txt consists of the six ASCII characters foobar. What is the output of the following program?
```c
int main(){
    int fd1, fd2;
    char c;
    fd1 = open(“foobar.txt”, O_RDONLY, 0);
    fd2 = open(“foobar.txt”, O_RDONLY, 0);
    read(fd1, &c, 1);
    read(fd2, &c, 1);
    printf(“c = %c\n”, c);
    exit(0);
} 
```

Output: 'f'

3.	Suppose the disk file foobar.txt consists of the six ASCII characters foobar. What is the output of the following program?
```c
int main(){
    int fd;
    char c;
    fd = open(“foobar.txt”, O_RDONLY, 0);
    if (fork()==0){
    	read(fd, &c, 1);
    	exit(0);
    }
    Wait(NULL);
     read(fd, &c, 1);
    printf(“c = %c\n”, c);
    exit(0);
} 
```

Output: c = 'o'